## Generating binary categories for training
**Steps**

- Setup categoricals for each segment based on author consensus
- Assign a binary value to each category of segment, in corresponding category columns (one-hot encoding)

In [1]:
import pandas as pd
import numpy as np
import nltk

In [2]:
# from sqlalchemy import create_engine
# from sqlalchemy_utils import database_exists, create_database
import psycopg2

In [3]:
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [4]:
#Connect to db
dbname = 'beforeiagree_db'
username = 'peterostendorp'

#Create engine
con = psycopg2.connect(database = dbname, user = username)

In [5]:
#Get annotations df
sql = """
SELECT * FROM annotations
WHERE "Policy UID" IN
(SELECT "Policy UID" FROM sites
WHERE "In 115 Set?" = TRUE);
"""
annotations = pd.read_sql_query(sql,con)

In [6]:
#Get sites df
sql = """
SELECT * FROM sites
WHERE "In 115 Set?" = TRUE;
"""
sites = pd.read_sql_query(sql,con)

In [7]:
#Get segments df
sql = """
SELECT * FROM segments
WHERE "Policy UID" IN
(SELECT "Policy UID" FROM sites
WHERE "In 115 Set?" = TRUE)
"""

segments = pd.read_sql_query(sql,con)

In [159]:
#df.groupby("date").agg({"duration": np.sum, "user_id": lambda x: x.nunique()})
#            duration  user_id
print(annotations['Policy UID'].nunique())
annotated_segments = pd.DataFrame(annotations.groupby('Policy UID').agg(lambda x: x.nunique())['segment_id'])
print(annotated_segments)
annotations.head()

115
            segment_id
Policy UID            
20                  36
21                  36
26                  68
32                  29
33                  65
58                  47
59                  13
70                  71
82                  46
93                  40
98                  41
105                 37
133                 43
135                 39
144                 31
164                 17
175                 34
186                 47
200                 43
202                 26
207                 34
228                 25
303                 54
320                 44
325                 20
331                 17
348                 62
359                 41
394                 42
414                 23
...                ...
1206                17
1221                 6
1224                19
1252                25
1259                56
1261                13
1264                14
1300                82
1306                26
1360                19
1361   

,Policy UID,annotation_id,batch_id,annotator_id,segment_id,category_name,attributes_value_pairs,date,policy_url
0,1017,20137,test_category_labeling_highlight_fordham_aaaaa,121,0,Other,"{""Other Type"": {""selectedText"": ""Sci-News.com ...",NaT,http://www.sci-news.com/privacy-policy.html
1,1017,20324,test_category_labeling_highlight_fordham_aaaaa,121,1,First Party Collection/Use,"{""Collection Mode"": {""selectedText"": ""nformati...",NaT,http://www.sci-news.com/privacy-policy.html
2,1017,20325,test_category_labeling_highlight_fordham_aaaaa,121,1,First Party Collection/Use,"{""Collection Mode"": {""selectedText"": ""nformati...",NaT,http://www.sci-news.com/privacy-policy.html
3,1017,20326,test_category_labeling_highlight_fordham_aaaaa,121,2,Data Retention,"{""Personal Information Type"": {""selectedText"":...",NaT,http://www.sci-news.com/privacy-policy.html
4,1017,20327,test_category_labeling_highlight_fordham_aaaaa,121,3,First Party Collection/Use,"{""Collection Mode"": {""selectedText"": ""Not sele...",NaT,http://www.sci-news.com/privacy-policy.html


In [201]:
print(segments['Policy UID'].nunique())
n_segments = pd.DataFrame(segments.groupby('Policy UID').count()['segment_id'])
print(n_segments['segment_id'].sum())
print(n_segments)
segments.head()

115
6469
            segment_id
Policy UID            
20                  36
21                  36
26                  68
32                 348
33                 585
58                  47
59                  13
70                 142
82                  92
93                  40
98                  41
105                 37
133                 43
135                 39
144                186
164                 17
175                 34
186                376
200                 43
202                 26
207                 68
228                150
303                 54
320                 44
325                 20
331                 17
348                 62
359                 41
394                 42
414                 23
...                ...
1206                17
1221                 6
1224                19
1252                25
1259                56
1261                13
1264                14
1300               164
1306                26
1360                19
13

,Policy UID,segment_id,segments
0,20,0,<strong> Privacy Policy </strong> <br> <br> <s...
1,20,1,This privacy policy does not apply to Sites ma...
2,20,2,"By visiting our Sites, you are accepting the p..."
3,20,3,<strong> What Information Is Collected? </stro...
4,20,4,<strong> Personally Identifiable Information <...


In [134]:
print(sites['Policy UID'].nunique())
print(sites['Site UID'].nunique())
sites.head()

115
115


,Policy UID,Site UID,Site URL,Site Human-Readable Name,Site Check Date,In 115 Set?,Comments,Sector,Policy URL,Policy collection date,Policy last updated date,policy_text
0,20,1,theatlantic.com,The Atlantic,2016-02-08,True,"Alexa Rank: 975 (Global), 289 (US)",Arts,theatlantic.com/privacy-policy/,2015-07-02,2015-01-01,<strong> Privacy Policy </strong> <br> <br> <s...
1,21,2,imdb.com,IMDb,2016-02-08,True,"Alexa Rank: 49 (Global), 27 (US)",Arts,imdb.com/privacy,2015-07-02,2014-12-05,"IMDb Privacy Notice <br> <br>|||Last Updated, ..."
2,26,3,nytimes.com,New York Times,2016-02-08,True,"Alexa Rank: 101 (Global), 22 (US)",Arts,nytimes.com/privacy,2015-07-08,2015-06-10,<strong> Privacy Policy </strong> <br> <br> La...
3,32,4,theverge.com,The Verge,2016-02-08,True,"Alexa Rank: 525 (Global), 230 (US)",Home,voxmedia.com/privacy-policy,2015-07-02,2014-05-01,Vox Media Privacy Policy <br> <br>|||<strong> ...
4,33,5,nbc.com,NBC Universal,2016-02-08,True,"Alexa Rank: 1548 (Global), 426 (US)",Arts,nbcuniversal.com/privacy/full-privacy-policy,2015-07-02,2015-01-14,Full Privacy Policy <br> <br> Last updated: 14...


There are many cases where the number of segments in a doc far exceeds the number of annotated segments.

In [166]:
#How do annotated segments compare to total number of segments?
for i in range(0,115):
    if annotated_segments.iloc[i]['segment_id'] == n_segments.iloc[i]['segment_id']:
        print('Yes')
    else:
        print('No')

Yes
Yes
Yes
No
No
Yes
Yes
No
No
Yes
Yes
Yes
Yes
Yes
No
Yes
Yes
No
Yes
Yes
No
No
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
No
Yes
Yes
Yes
No
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
No
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
No
Yes
No
No
Yes
No
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
No
Yes
Yes
No
Yes
Yes
Yes
Yes
Yes
Yes
Yes
No
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
No
No
Yes


## This is the tricky bit where we join segments and annotations

In [8]:
#Indexed version of segments
seg_ind = segments.set_index(['Policy UID','segment_id'])
seg_ind['key'] = 1 #TFH
seg_ind.head()

segments  key
Policy UID segment_id                                                        
20         0           <strong> Privacy Policy </strong> <br> <br> <s...    1
           1           This privacy policy does not apply to Sites ma...    1
           2           By visiting our Sites, you are accepting the p...    1
           3           <strong> What Information Is Collected? </stro...    1
           4           <strong> Personally Identifiable Information <...    1

In [9]:
#Indexed version of annotations
ann_ind = annotations.set_index(['Policy UID','segment_id'])
ann_ind['key'] = 1 #TFH
ann_ind.head()

annotation_id  \
Policy UID segment_id                  
1017       0                   20137   
           1                   20324   
           1                   20325   
           2                   20326   
           3                   20327   

                                                             batch_id  \
Policy UID segment_id                                                   
1017       0           test_category_labeling_highlight_fordham_aaaaa   
           1           test_category_labeling_highlight_fordham_aaaaa   
           1           test_category_labeling_highlight_fordham_aaaaa   
           2           test_category_labeling_highlight_fordham_aaaaa   
           3           test_category_labeling_highlight_fordham_aaaaa   

                       annotator_id               category_name  \
Policy UID segment_id                                             
1017       0                    121                       Other   
           1                    121  First Party Collection/Use   
           1                    121  First Party Collection/Use   
           2                    121              Data Retention   
           3                    121  First Party Collection/Use   

                                                  attributes_value_pairs date  \
Policy UID segment_id                                                           
1017       0           {"Other Type": {"selectedText": "Sci-News.com ...  NaT   
           1           {"Collection Mode": {"selectedText": "nformati...  NaT   
           1           {"Collection Mode": {"selectedText": "nformati...  NaT   
           2           {"Personal Information Type": {"selectedText":...  NaT   
           3           {"Collection Mode": {"selectedText": "Not sele...  NaT   

                                                        policy_url  key  
Policy UID segment_id                                                    
1017       0           http://www.sci-news.com/privacy-policy.html    1  
           1           http://www.sci-news.com/privacy-policy.html    1  
           1           http://www.sci-news.com/privacy-policy.html    1  
           2           http://www.sci-news.com/privacy-policy.html    1  
           3           http://www.sci-news.com/privacy-policy.html    1

In [ ]:
#Join the annotations with the segments using an outer join
#joined = pd.merge(annotations,segments,on=['Policy UID','segment_id'],how='outer')
joined = seg_ind.merge(ann_ind)
print(joined.shape)
joined.head()

There are usually instances where a single segment gets multiple categories.

In [173]:
#Are there often cases where there are multiple category values for a single segment?
print(joined.groupby(['Policy UID','segment_id']).agg(lambda x: x.nunique())['category_name'])

Policy UID  segment_id
20          0             1
            1             1
            2             2
            3             1
            4             2
            5             1
            6             2
            7             1
            8             1
            9             2
            10            1
            11            2
            12            1
            13            1
            14            2
            15            1
            16            2
            17            2
            18            1
            19            1
            20            1
            21            2
            22            2
            23            3
            24            1
            25            1
            26            2
            27            2
            28            1
            29            1
                         ..
1713        59            2
            60            3
            61            1
            62           

In [14]:
pids = list(segments['Policy UID'].unique())
categories = pd.DataFrame(columns=['Policy UID','segment_id','category_name'])
for pid in pids:
    df = segments[segments['Policy UID']==pid]
    segs = list(df['segment_id'].unique())
    for seg in segs:
        ann = annotations[(annotations['Policy UID']==pid) & annotations['segment_id']==seg]['category_name']
        cat = ann if ann.shape[0] != 0 else 'None'
        print('Policy: ' + str(pid) + ', Segment: ' + str(seg) + ', Category: ' + str(cat))
        tmp = pd.DataFrame({'Policy UID':[pid], 'segment_id':[seg], 'category_name':[cat]})
        categories = pd.concat([categories,tmp],ignore_index=True)

Policy: 20, Segment: 0, Category: 0                                       Other
1                  First Party Collection/Use
2                  First Party Collection/Use
3                              Data Retention
4                  First Party Collection/Use
5                  First Party Collection/Use
6                  First Party Collection/Use
7                  First Party Collection/Use
8        International and Specific Audiences
9                                       Other
10                 First Party Collection/Use
11                                      Other
12                                      Other
13                        User Choice/Control
14             Third Party Sharing/Collection
15                                      Other
16                        User Choice/Control
17                        User Choice/Control
18             User Access, Edit and Deletion
19                                      Other
20                                      Other


Policy: 26, Segment: 38, Category: None
Policy: 26, Segment: 39, Category: None
Policy: 26, Segment: 40, Category: None
Policy: 26, Segment: 41, Category: None
Policy: 26, Segment: 42, Category: None
Policy: 26, Segment: 43, Category: None
Policy: 26, Segment: 44, Category: None
Policy: 26, Segment: 45, Category: None
Policy: 26, Segment: 46, Category: None
Policy: 26, Segment: 47, Category: None
Policy: 26, Segment: 48, Category: None
Policy: 26, Segment: 49, Category: None
Policy: 26, Segment: 50, Category: None
Policy: 26, Segment: 51, Category: None
Policy: 26, Segment: 52, Category: None
Policy: 26, Segment: 53, Category: None
Policy: 26, Segment: 54, Category: None
Policy: 26, Segment: 55, Category: None
Policy: 26, Segment: 56, Category: None
Policy: 26, Segment: 57, Category: None
Policy: 26, Segment: 60, Category: None
Policy: 26, Segment: 61, Category: None
Policy: 26, Segment: 62, Category: None
Policy: 26, Segment: 63, Category: None
Policy: 26, Segment: 64, Category: None


Policy: 33, Segment: 40, Category: None
Policy: 33, Segment: 41, Category: None
Policy: 33, Segment: 42, Category: None
Policy: 33, Segment: 43, Category: None
Policy: 33, Segment: 44, Category: None
Policy: 33, Segment: 46, Category: None
Policy: 33, Segment: 48, Category: None
Policy: 33, Segment: 49, Category: None
Policy: 33, Segment: 50, Category: None
Policy: 33, Segment: 51, Category: None
Policy: 33, Segment: 52, Category: None
Policy: 33, Segment: 53, Category: None
Policy: 33, Segment: 54, Category: None
Policy: 33, Segment: 55, Category: None
Policy: 33, Segment: 57, Category: None
Policy: 33, Segment: 58, Category: None
Policy: 33, Segment: 59, Category: None
Policy: 33, Segment: 60, Category: None
Policy: 33, Segment: 61, Category: None
Policy: 33, Segment: 62, Category: None
Policy: 21, Segment: 0, Category: 0                                       Other
1                  First Party Collection/Use
2                  First Party Collection/Use
3                           

Policy: 32, Segment: 9, Category: None
Policy: 32, Segment: 10, Category: None
Policy: 32, Segment: 11, Category: None
Policy: 32, Segment: 12, Category: None
Policy: 32, Segment: 13, Category: None
Policy: 32, Segment: 14, Category: None
Policy: 32, Segment: 15, Category: None
Policy: 32, Segment: 16, Category: None
Policy: 32, Segment: 17, Category: None
Policy: 32, Segment: 19, Category: None
Policy: 32, Segment: 20, Category: None
Policy: 32, Segment: 21, Category: None
Policy: 32, Segment: 22, Category: None
Policy: 32, Segment: 23, Category: None
Policy: 32, Segment: 24, Category: None
Policy: 32, Segment: 25, Category: None
Policy: 32, Segment: 26, Category: None
Policy: 32, Segment: 27, Category: None
Policy: 32, Segment: 28, Category: None
Policy: 70, Segment: 23, Category: None
Policy: 70, Segment: 37, Category: None
Policy: 70, Segment: 22, Category: None
Policy: 70, Segment: 19, Category: None
Policy: 70, Segment: 20, Category: None
Policy: 70, Segment: 21, Category: None
P

Policy: 70, Segment: 53, Category: None
Policy: 70, Segment: 54, Category: None
Policy: 70, Segment: 55, Category: None
Policy: 70, Segment: 56, Category: None
Policy: 70, Segment: 57, Category: None
Policy: 70, Segment: 58, Category: None
Policy: 70, Segment: 59, Category: None
Policy: 70, Segment: 60, Category: None
Policy: 70, Segment: 61, Category: None
Policy: 70, Segment: 62, Category: None
Policy: 70, Segment: 63, Category: None
Policy: 70, Segment: 64, Category: None
Policy: 70, Segment: 65, Category: None
Policy: 70, Segment: 66, Category: None
Policy: 70, Segment: 67, Category: None
Policy: 70, Segment: 68, Category: None
Policy: 70, Segment: 69, Category: None
Policy: 70, Segment: 70, Category: None
Policy: 59, Segment: 6, Category: None
Policy: 59, Segment: 5, Category: None
Policy: 59, Segment: 0, Category: 0                                       Other
1                  First Party Collection/Use
2                  First Party Collection/Use
3                             

...

Policy: 1539, Segment: 2, Category: None
Policy: 1539, Segment: 3, Category: None
Policy: 1539, Segment: 4, Category: None
Policy: 1539, Segment: 5, Category: None
Policy: 1539, Segment: 6, Category: None
Policy: 1539, Segment: 7, Category: None
Policy: 1539, Segment: 8, Category: None
Policy: 1539, Segment: 9, Category: None
Policy: 1539, Segment: 10, Category: None
Policy: 1539, Segment: 11, Category: None
Policy: 1539, Segment: 12, Category: None
Policy: 1539, Segment: 14, Category: None
Policy: 1539, Segment: 15, Category: None
Policy: 1539, Segment: 16, Category: None
Policy: 1539, Segment: 17, Category: None
Policy: 1539, Segment: 18, Category: None
Policy: 1539, Segment: 19, Category: None
Policy: 1539, Segment: 20, Category: None
Policy: 1539, Segment: 21, Category: None
Policy: 1539, Segment: 22, Category: None
Policy: 1539, Segment: 23, Category: None
Policy: 1539, Segment: 24, Category: None
Policy: 1539, Segment: 25, Category: None
Policy: 1539, Segment: 26, Category: None


Policy: 1545, Segment: 17, Category: None
Policy: 1545, Segment: 18, Category: None
Policy: 1545, Segment: 19, Category: None
Policy: 1545, Segment: 20, Category: None
Policy: 1545, Segment: 21, Category: None
Policy: 1545, Segment: 22, Category: None
Policy: 1545, Segment: 23, Category: None
Policy: 1545, Segment: 24, Category: None
Policy: 1545, Segment: 25, Category: None
Policy: 1545, Segment: 26, Category: None
Policy: 1545, Segment: 27, Category: None
Policy: 1618, Segment: 0, Category: 0                                       Other
1                  First Party Collection/Use
2                  First Party Collection/Use
3                              Data Retention
4                  First Party Collection/Use
5                  First Party Collection/Use
6                  First Party Collection/Use
7                  First Party Collection/Use
8        International and Specific Audiences
9                                       Other
10                 First Party Collection/

Policy: 1673, Segment: 17, Category: None
Policy: 1673, Segment: 18, Category: None
Policy: 1673, Segment: 19, Category: None
Policy: 1673, Segment: 20, Category: None
Policy: 1673, Segment: 21, Category: None
Policy: 1673, Segment: 22, Category: None
Policy: 1673, Segment: 23, Category: None
Policy: 1673, Segment: 24, Category: None
Policy: 1673, Segment: 25, Category: None
Policy: 1673, Segment: 26, Category: None
Policy: 1673, Segment: 27, Category: None
Policy: 1673, Segment: 28, Category: None
Policy: 1673, Segment: 29, Category: None
Policy: 1673, Segment: 30, Category: None
Policy: 1673, Segment: 31, Category: None
Policy: 1673, Segment: 32, Category: None
Policy: 1673, Segment: 34, Category: None
Policy: 1673, Segment: 35, Category: None
Policy: 1673, Segment: 36, Category: None
Policy: 1673, Segment: 37, Category: None
Policy: 1673, Segment: 38, Category: None
Policy: 1673, Segment: 39, Category: None
Policy: 1673, Segment: 40, Category: None
Policy: 1673, Segment: 41, Categor

Policy: 1637, Segment: 5, Category: None
Policy: 1637, Segment: 6, Category: None
Policy: 1637, Segment: 7, Category: None
Policy: 1637, Segment: 8, Category: None
Policy: 1637, Segment: 9, Category: None
Policy: 1637, Segment: 10, Category: None
Policy: 1637, Segment: 11, Category: None
Policy: 1637, Segment: 12, Category: None
Policy: 1637, Segment: 13, Category: None
Policy: 1637, Segment: 14, Category: None
Policy: 1666, Segment: 0, Category: 0                                       Other
1                  First Party Collection/Use
2                  First Party Collection/Use
3                              Data Retention
4                  First Party Collection/Use
5                  First Party Collection/Use
6                  First Party Collection/Use
7                  First Party Collection/Use
8        International and Specific Audiences
9                                       Other
10                 First Party Collection/Use
11                                      Oth

Policy: 1683, Segment: 16, Category: None
Policy: 1683, Segment: 17, Category: None
Policy: 1683, Segment: 18, Category: None
Policy: 1683, Segment: 19, Category: None
Policy: 1683, Segment: 20, Category: None
Policy: 1683, Segment: 21, Category: None
Policy: 1683, Segment: 22, Category: None
Policy: 1683, Segment: 23, Category: None
Policy: 1683, Segment: 24, Category: None
Policy: 1683, Segment: 25, Category: None
Policy: 1683, Segment: 26, Category: None
Policy: 1683, Segment: 27, Category: None
Policy: 1683, Segment: 28, Category: None
Policy: 1683, Segment: 29, Category: None
Policy: 1683, Segment: 30, Category: None
Policy: 1683, Segment: 31, Category: None
Policy: 1683, Segment: 32, Category: None
Policy: 1703, Segment: 0, Category: 0                                       Other
1                  First Party Collection/Use
2                  First Party Collection/Use
3                              Data Retention
4                  First Party Collection/Use
5                  F

This mess above yields the same answer as when using the join and agg method below.

In [17]:
categories.head()

,Policy UID,category_name,segment_id
0,20,0 Other ...,0
1,20,11178 Data Retention ...,1
2,20,None,2
3,20,None,3
4,20,None,4


In [106]:
#Set up table in which categories are assigned to each unique document segment. Those that aren't
#tagged are labeled as "None".
#Join category name to segments and determine the mode of each category, concat back onto original segments
tmp = pd.merge(segments,annotations,on=['Policy UID','segment_id'],how='outer')
tmp['category_name'].fillna(value='None')
tmp2 = pd.DataFrame(tmp.groupby(by=['Policy UID','segment_id']).count())
print(tmp2.shape)
tmp2.head(100)#['category_name'].agg(lambda x: x.value_counts().index[0]))

(3792, 8)


segments  annotation_id  batch_id  annotator_id  \
Policy UID segment_id                                                    
20         0                  3              3         3             3   
           1                  3              3         3             3   
           2                  7              7         7             7   
           3                 13             13        13            13   
           4                  5              5         5             5   
           5                 16             16        16            16   
           6                 10             10        10            10   
           7                  7              7         7             7   
           8                  4              4         4             4   
           9                  7              7         7             7   
           10                 9              9         9             9   
           11                15             15        15            15   
           12                 3              3         3             3   
           13                20             20        20            20   
           14                 3              3         3             3   
           15                 2              2         2             2   
           16                12             12        12            12   
           17                 6              6         6             6   
           18                 3              3         3             3   
           19                 5              5         5             5   
           20                 5              5         5             5   
           21                 4              4         4             4   
           22                 4              4         4             4   
           23                 7              7         7             7   
           24                 3              3         3             3   
           25                 3              3         3             3   
           26                 3              3         3             3   
           27                 3              3         3             3   
           28                 3              3         3             3   
           29                 6              6         6             6   
...                         ...            ...       ...           ...   
21         34                 2              2         2             2   
           35                 6              6         6             6   
26         0                  3              3         3             3   
           1                  3              3         3             3   
           2                  3              3         3             3   
           3                  3              3         3             3   
           4                  3              3         3             3   
           5                  3              3         3             3   
           6                  3              3         3             3   
           7                  3              3         3             3   
           8                  3              3         3             3   
           9                 12             12        12            12   
           10                 3              3         3             3   
           11                 9              9         9             9   
           12                 4              4         4             4   
           13                 9              9         9             9   
           14                 3              3         3             3   
           15                 9              9         9             9   
           16                10             10        10            10   
           17                 4              4         4             4   
           18                 3              3         3             3   
           19              

In [85]:
tmp.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 40133 entries, 0 to 40132
Data columns (total 10 columns):
Policy UID                40133 non-null int64
segment_id                40133 non-null int64
segments                  40133 non-null object
annotation_id             40133 non-null int64
batch_id                  40133 non-null object
annotator_id              40133 non-null int64
category_name             40133 non-null object
attributes_value_pairs    40133 non-null object
date                      36399 non-null datetime64[ns]
policy_url                40133 non-null object
dtypes: datetime64[ns](1), int64(4), object(5)
memory usage: 3.4+ MB


In [60]:
categories = list(tmp2['category_name'].unique())
cols = {'Other': 'other',
        'Policy Change': 'policy_change',
        'First Party Collection/Use': 'first_party_collection_use',
        'Third Party Sharing/Collection': 'third_party_sharing_collection',
        'Do Not Track': 'do_not_track',
        'User Choice/Control': 'user_choice_control',
        'International and Specific Audiences': 'international_specific_audiences',
        'Data Security': 'data_security',
        'Data Retention': 'data_retention',
        'User Access, Edit and Deletion': 'user_access_edit_deletion'}

## Segment-level categorization
Loop through categories and generate one-hot encoding of each category and stick them into new columns with the corresponding names in cols

In [61]:
#Loop through the categories and generate a set of new columns with names in cols
binary_categories = pd.DataFrame()

for category in categories:
    one_hot = lambda s: 1 if s.startswith(category) else 0
    binary_categories[cols[category]] = tmp2['category_name'].apply(one_hot)

In [65]:
binary_categories.head()

other  policy_change  first_party_collection_use  \
Policy UID segment_id                                                     
20         0               1              0                           0   
           1               1              0                           0   
           2               0              1                           0   
           3               0              0                           1   
           4               0              0                           1   

                       third_party_sharing_collection  do_not_track  \
Policy UID segment_id                                                 
20         0                                        0             0   
           1                                        0             0   
           2                                        0             0   
           3                                        0             0   
           4                                        0             0   

                       user_choice_control  international_specific_audiences  \
Policy UID segment_id                                                          
20         0                             0                                 0   
           1                             0                                 0   
           2                             0                                 0   
           3                             0                                 0   
           4                             0                                 0   

                       data_security  data_retention  \
Policy UID segment_id                                  
20         0                       0               0   
           1                       0               0   
           2                       0               0   
           3                       0               0   
           4                       0               0   

                       user_access_edit_deletion  
Policy UID segment_id                             
20         0                                   0  
           1                                   0  
           2                                   0  
           3                                   0  
           4                                   0

In [27]:
#Create engine for persisting
from sqlalchemy import create_engine
from sqlalchemy_utils import database_exists, create_database

In [28]:
engine = create_engine('postgres://%s@localhost/%s'%(username,dbname))
print(engine.url)

postgres://peterostendorp@localhost/beforeiagree_db


In [29]:
binary_categories.to_sql('binary_segment_categories',engine, if_exists='replace')

## Policy-level categorization

In [39]:
binary_categories_policy = binary_categories.reset_index(level=[0,1]).groupby('Policy UID').sum().drop('segment_id',axis=1)
binary_categories_policy = binary_categories_policy.applymap(lambda x: 1 if x > 0 else 0)
binary_categories_policy.head()

,other,policy_change,first_party_collection_use,third_party_sharing_collection,do_not_track,user_choice_control,international_specific_audiences,data_security,data_retention,user_access_edit_deletion
Policy UID,,,,,,,,,,
20,1,1,1,1,1,1,1,1,1,1
21,1,1,1,1,0,1,1,1,0,1
26,1,1,1,1,0,1,1,1,0,1
32,1,1,1,1,0,1,1,1,1,0
33,1,1,1,1,0,1,1,1,1,1


In [40]:
binary_categories_policy.sum()

other                               112
policy_change                        83
first_party_collection_use          112
third_party_sharing_collection      110
do_not_track                         22
user_choice_control                  98
international_specific_audiences     85
data_security                        94
data_retention                       25
user_access_edit_deletion            73
dtype: int64

Most policies contain info on:
- Other
- First and third party data collection
- user choice control
- data security

Less frequently mentioned:
- data retention
- user access edit deletion
- do not track

In [41]:
binary_categories_policy.to_sql('binary_policy_categories',engine)